# 导入相关库

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Duplicate key in file PosixPath('/Users/chengxin/opt/anaconda3/lib/python3.8/site-packages/matplotlib/mpl-data/matplotlibrc'), line 758 ('font.family\t    : sans-serif')
Duplicate key in file PosixPath('/Users/chengxin/opt/anaconda3/lib/python3.8/site-packages/matplotlib/mpl-data/matplotlibrc'), line 759 ('font.sans-serif\t    : SimHei')
Duplicate key in file PosixPath('/Users/chengxin/opt/anaconda3/lib/python3.8/site-packages/matplotlib/mpl-data/matplotlibrc'), line 760 ('axes.unicode_minus   : False')


# 导入数据 & 检查数据

In [2]:
testAB = pd.read_csv("../data/ab_data.csv")

In [3]:
# check the dataset
testAB.head(5)

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [4]:
# check the dataset
testAB.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294478 non-null  int64 
 1   timestamp     294478 non-null  object
 2   group         294478 non-null  object
 3   landing_page  294478 non-null  object
 4   converted     294478 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


- 数据有5个维度
- 无缺失值

# 数据清洗

- 数据清洗之前先确保数据正确性，包括交叉验证有无数据属性间匹配错误
- 删除无效数据，生成相关数据

In [5]:
testAB[(testAB['group']=="control")&(testAB['landing_page']!="old_page")]
testAB[(testAB['group']=="treatment")&(testAB['landing_page']!="new_page")]

,user_id,timestamp,group,landing_page,converted
308,857184,2017-01-20 07:34:59.832626,treatment,old_page,0
327,686623,2017-01-09 14:26:40.734775,treatment,old_page,0
357,856078,2017-01-12 12:29:30.354835,treatment,old_page,0
685,666385,2017-01-23 08:11:54.823806,treatment,old_page,0
713,748761,2017-01-10 15:47:44.445196,treatment,old_page,0
...,...,...,...,...,...
293773,688144,2017-01-16 20:34:50.450528,treatment,old_page,1
293817,876037,2017-01-17 16:15:08.957152,treatment,old_page,1
293917,738357,2017-01-05 15:37:55.729133,treatment,old_page,0
294014,813406,2017-01-09 06:25:33.223301,treatment,old_page,0


In [6]:
# 删除错误的数据
testABCleaned = testAB[((testAB['group']=='control')&(testAB['landing_page']=='old_page'))|((testAB['group']=='treatment')&(testAB['landing_page']=='new_page'))]
# 检查是否有空值
testABCleaned.isnull().sum()

user_id         0
timestamp       0
group           0
landing_page    0
converted       0
dtype: int64

In [7]:
# 删除重复的值
testABCleaned = testABCleaned.drop_duplicates(subset='user_id',keep = 'first')
# 重置索引
testABCleaned = testABCleaned.reset_index()

In [8]:
# 查看新旧页面流量所占比例，是否平均
proportion = len(testABCleaned[testABCleaned['group'] =='treatment'])/len(testABCleaned)*100
proportion

50.00619442226688

In [10]:
testABCleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290584 entries, 0 to 290583
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   index         290584 non-null  int64 
 1   user_id       290584 non-null  int64 
 2   timestamp     290584 non-null  object
 3   group         290584 non-null  object
 4   landing_page  290584 non-null  object
 5   converted     290584 non-null  int64 
 6   date          290584 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 15.5+ MB


# 数据分析

In [11]:
import datetime
#根据timestamp 生成新的特征date
testABCleaned = testABCleaned.astype({"timestamp":"datetime64[ns]"})
testABCleaned_new = testABCleaned
testABCleaned['date'] = 0
for index, row in testABCleaned.iterrows():
    currentDate = row['date']
    values = row['timestamp']
    values = values.strftime('%Y-%m-%d')
    testABCleaned_new.at[index,currentDate] = values
testABCleaned_new

,index,user_id,timestamp,group,landing_page,converted,date,0
0,0,851104,2017-01-21 22:11:48.556739,control,old_page,0,0,2017-01-21
1,1,804228,2017-01-12 08:01:45.159739,control,old_page,0,0,2017-01-12
2,2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0,0,2017-01-11
3,3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0,0,2017-01-08
4,4,864975,2017-01-21 01:52:26.210827,control,old_page,1,0,2017-01-21
...,...,...,...,...,...,...,...,...
290579,294473,751197,2017-01-03 22:28:38.630509,control,old_page,0,0,2017-01-03
290580,294474,945152,2017-01-12 00:51:57.078372,control,old_page,0,0,2017-01-12
290581,294475,734608,2017-01-22 11:45:03.439544,control,old_page,0,0,2017-01-22
290582,294476,697314,2017-01-15 01:20:28.957438,control,old_page,0,0,2017-01-15


In [12]:
testABCleaned_new = testABCleaned_new.drop(['date'],axis=1)
testABCleaned_new.columns

Index(['index', 'user_id', 'timestamp', 'group', 'landing_page', 'converted',
       0],
      dtype='object')

In [13]:
testABCleaned_new = testABCleaned_new.rename(columns = {0:'date'}) #把列名F改为hahahah

In [14]:
table1 = pd.pivot_table(testABCleaned_new,index = ["landing_page","date"],values=["user_id"],aggfunc={"landing_page":"count","date":"count"})
#table1[(table1['landing_page']=='new_page')&(table1['date']=='2017-01-02')]
newPageDate = table1.loc['new_page']['date']
oldPageDate = table1.loc['old_page']['date']
oldPageDate.values

array([2859, 6590, 6578, 6427, 6606, 6604, 6687, 6628, 6654, 6688, 6522,
       6552, 6548, 6714, 6591, 6617, 6482, 6578, 6534, 6749, 6596, 6716,
       3754])

In [15]:
table3 = pd.pivot_table(testABCleaned_new,index = ["date"],values=["user_id"],aggfunc={"date":"count"})
table3

,date
date,
2017-01-02,5712
2017-01-03,13208
2017-01-04,13119
2017-01-05,12932
2017-01-06,13353
2017-01-07,13213
2017-01-08,13387
2017-01-09,13243
2017-01-10,13350


In [16]:
import pyecharts.options as opts
from pyecharts.charts import Line
from pyecharts.faker import Faker


line = (
    Line()
    .add_xaxis(oldPageDate.index.tolist())
    .add_yaxis("旧页面流量", oldPageDate.values.tolist(), is_connect_nones=True)
    .add_yaxis("新页面流量", newPageDate.values.tolist(), is_connect_nones=True)
    #.is_hover_animation: bool = True,
    .set_global_opts(title_opts=opts.TitleOpts(title="Line-连接空数据",title_link="https://www.baidu.com/"))
    .set_global_opts(toolbox_opts=opts.ToolboxOpts(is_show = True))
    .set_global_opts(yaxis_opts=opts.AxisOpts(min_ = "dataMin"))
    
    #.set_global_opts(toolbox_opts=opts.ToolBoxFeatureOpts(save_as_image=ToolBoxFeatureSaveAsImageOpts(is_show=True)))
    
    #.render("line_connect_null.html")
    #.LineItem()
)


In [17]:
line.load_javascript()
line.render_notebook()

- 因为两种总体转化率未知，样本量大，样本转化率方差，转化率已知，采用Z检验

In [19]:
# 旧版、新版用户数（query是条件过滤函数）
n_old = testABCleaned.query('group=="control"').shape[0]
n_new = testABCleaned.query('group=="treatment"').shape[0]
# 旧版、新版转化用户数
convert_old = testABCleaned.query('group=="control" & converted==1').shape[0]
convert_new = testABCleaned.query('group=="treatment" & converted==1').shape[0]
# 旧版、新版转化率
p_old = convert_old / n_old
p_new = convert_new / n_new
p_c = (convert_old + convert_new)/(n_old + n_new)
z = (p_old - p_new)/ np.sqrt(p_c*(1 - p_c)*( 1/n_old + 1/n_new))
from scipy.stats import norm
z_alpha = norm.ppf(0.05)
# 若为双侧，则norm.ppf(0.05/2)
print('旧版总受试用户数:', n_old, '旧版转化用户数:', convert_old, '旧版转化率:', p_old)
print('新版总受试用户数:', n_new, '新版转化用户数:', convert_new, '新版转化率:', p_new)
print('转化率的联合估计:', p_c)
print('检验统计量z:', z)
print(z_alpha)
result = "落入拒绝域，拒绝零假设" if abs(z) > abs(z_alpha) else "接受零假设"
print(result)

旧版总受试用户数: 145274 旧版转化用户数: 17489 旧版转化率: 0.1203863045004612
新版总受试用户数: 145310 新版转化用户数: 17872 新版转化率: 0.12299222352212512
转化率的联合估计: 0.12168942543292129
检验统计量z: -2.1484056695589
-1.6448536269514729
落入拒绝域，拒绝零假设


In [ ]:
- 合并标准差

In [20]:
# 合并标准差
std_old = testABCleaned[testABCleaned.landing_page=="old_page"].converted.std()
std_new = testABCleaned[testABCleaned.landing_page=="new_page"].converted.std()
s = np.sqrt(((n_old - 1)* std_old**2 + (n_new - 1)* std_new**2 ) / (n_old + n_new - 2))
# 效应量Cohen's d
d = (p_old - p_new) / s
print('Cohen\'s d为：', d)

Cohen's d为： -0.007970992391336005
